# Introduction: Modeling

The last step in the machine learning pipeline is also where the value is derived. After we have developed the labels, each with its own cutoff time, we need to train a model to map from the features to predict the label. 

In this notebook, we will use the feature matrices and label times to train and test a machine learning model. We will work through a single prediction problem, predicting on the first of each month which customers will churn during the month with churn defined as no active membership for more than 31 days, a lead time of 1 month, and a prediction window of 1 month.

The general process of modeling is shown below:

![](../images/modeling_process.png)

## Approach

Our basic machine learning approach is:

1. Prepare data for machine learning
    * Fill in missing values with median imputation
    * Encoding of categorical values
2. Split data into training and hold out testing based on time 
3. Evaluate a baseline logistic regression model
    * Also try a naive baseline for comparison
4. Try a non-linear more capable classifier, the Random Forest 
    * Use mostly default hyperparameters
    * Evaluate on hold-out testing data
5. Inspect predictions to determine if business need has been met
    * Precision recall curve used to tune threshold
    * Confusion matrix to assess predictions
    * Determine business value
6. Optimizer model automatically using an auto-ml library
    * Using TPOT although many options exist
    
The final outcome is an optimized model that solves the business problem of predicting customer churn with given parameters. The model can be deployed - used to make predictions on new data

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

PARTITION_DIR = 's3://customer-churn-spark'

In [ ]:
p70_fm = pd.read_csv(f'{PARTITION_DIR}/p70/MS-31_feature_matrix.csv')
p70_fm.head()

### Retrieve Data

All of the data is stored in AWS S3. We'll retrieve 50 partitions of feature matrices. For this problem, we'll use a lead offset of 1 month meaning that our predictions are made one month in advance. Adjsting this value significantly affects the performance because it is harder for the model to learn meaningful relationships farther out from the end of the features (which are determined by the cutoff time).

In [ ]:
def retrieve_data(partition_num, label_type='MS-31'):
    """Retrieve features and labels and merge. Lead periods is the number of offsets"""

    # Read in features and labels
    fm = pd.read_csv(f'{PARTITION_DIR}/p{partition_num}/{label_type}_feature_matrix.csv', low_memory=False).\
        drop(columns=['label', 'days_to_churn', 'churn_date']).rename(
            columns={'time': 'cutoff_time'})
    labels = pd.read_csv(
        f'{PARTITION_DIR}/p{partition_num}/{label_type}_labels.csv', low_memory=False)

    # Merge together features and labels
    feature_matrix = fm.merge(labels, on=[
                              'msno', 'cutoff_time'], how='inner').sort_values(['msno', 'cutoff_time'])

    return feature_matrix

In [ ]:
fm = retrieve_data(50)
fm.head()

The next cell retrieves a number of feature matrices for training and for testing.

In [ ]:
import random
random.seed(50)

# Set number of train and testing feature matrices
fms_to_get = 50

# Choose random sample of partitions
ps = random.sample(list(range(900)), fms_to_get)

# Retrieve feature matrices from S3
fms = []
for i, r in enumerate(ps):
    fm = retrieve_data(r)
    print(f'{round(100 * (i / fms_to_get), 2)}% complete.', end='\r')
    if fm.shape[1] == 253:
        fms.append(fm)

# Join together and drop rows with unknown label
feature_matrix = pd.concat(fms)
feature_matrix = feature_matrix[~feature_matrix['label'].isna()].sort_values([
    'msno', 'cutoff_time'])

# Drop rows with no previous transactions
feature_matrix = feature_matrix[~feature_matrix['TIME_SINCE_LAST(transactions.transaction_date)'].isna(
)]

# Remove outliers
feature_matrix = feature_matrix[feature_matrix[
    'TOTAL_PREVIOUS_MONTH(transactions.planned_daily_price, transaction_date)'] < 10]
feature_matrix.shape

The cell below converts the boolean types to integers for use in a machine learning model. Most of the boolean indicate whether or not all the values for customer were True (`All` primitive) or if the date was a weekend (`IsWeekend` primitive).

In [ ]:
feature_matrix = feature_matrix.replace({'False': 0, 'True': 1})

In [ ]:
feature_matrix_original = feature_matrix.copy()
feature_matrix.drop(columns=[c for c in ['churn', 'days_to_next_churn',
                                         'churn_date'] if c in feature_matrix],
                    inplace=True)

bool_cols = [c for c in feature_matrix if 'ALL' in c or (
    'WEEKEND' in c and 'PERCENT_TRUE' not in c)]

for c in bool_cols:
    feature_matrix[c] = feature_matrix[c].astype(float)
feature_matrix[bool_cols].head()

In [ ]:
feature_matrix = feature_matrix.drop_duplicates(subset = ['msno', 'cutoff_time'])

## Data Cleaning

We'll do a few basic data cleaning steps:

* Remove columns with many missing values
* Remove columns with a single unique value
* Remove highly correlated - colinear - columns

#### Missing Values

We'll drop any columns with more than 90% missing values.

In [ ]:
missing_pct = feature_matrix.isnull().sum() / len(feature_matrix)
to_drop = list((missing_pct[missing_pct > 0.9]).index)
to_drop = [x for x in to_drop if x != 'days_to_churn']
to_drop

In [ ]:
feature_matrix.drop(columns=to_drop, inplace=True)
feature_matrix.shape

#### One Unique Value

Columns with only a single unique value contain no information and hence can be dropped.

In [ ]:
one_unique = feature_matrix.apply(lambda x: x.nunique() == 1, axis=0)
to_drop = list(one_unique[one_unique == True].index)
to_drop

In [ ]:
feature_matrix.drop(columns=to_drop, inplace=True)
feature_matrix.shape

#### Highly Correlated (collinear) Columns

Collinear columns can slow down training, lead to less interpretable models, and decrease generalization performance. Therefore, it's generally a good idea to remove one of each pair of highly correlated columns for machine learning. The following code identifies columns that exceed an absolute magnitude correlation of 0.95.

In [ ]:
threshold = 0.95

# Calculate correlations
corr_matrix = feature_matrix.corr().abs()

# Subset to the upper triangle of correlation matrix
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Identify names of columns with correlation above threshold
to_drop = [column for column in upper.columns if any(
    upper[column] >= threshold)]

In [ ]:
print(
    f'There are {len(to_drop)} columns to drop with correlation > {threshold}')

In [ ]:
feature_matrix.drop(columns=to_drop, inplace=True)
feature_matrix.shape

These data cleaning operations should increase the generalization performance of our model and make it more interpretable. A few simple operations can greatly improve a machine learning model and often are more effective than model optimization.

# Separate into Training and Testing Set

We'll separate into a training and testing set based on the date. We'll use 25% of the data in the testing and 75% in the training. Separating training and testing by the date is important in time sensitive problems because it prevents data leakage and gives a better estimate of the generalization performance of the model. On real data, our model will have to make forecasts of the future, and we can try to recreate that situation by using a hold-out set from later in time than the training data.

In [ ]:
feature_matrix['cutoff_time'] = pd.to_datetime(feature_matrix['cutoff_time'])
feature_matrix['cutoff_time'].describe()

The cell below splits the data based on the time. We'll try to use about 30% of the data for testing and the rest for training.

In [ ]:
split_date = pd.datetime(2016, 8, 1)


train = feature_matrix.loc[feature_matrix['cutoff_time'] < split_date].copy()
test = feature_matrix.loc[feature_matrix['cutoff_time'] >= split_date].copy()

train.sort_values(['cutoff_time'], inplace=True)
test.sort_values(['cutoff_time'], inplace=True)
train.shape, test.shape

In [ ]:
len(train) / len(feature_matrix)

# Feature Preparation

The next blocks of code get the features ready for machine learning.


### Encoding Categoricals

First we need to one hot encode the features. After doing this, we align the training and testing dataframes so they have the same columns.

In [ ]:
train = pd.get_dummies(train.drop(columns=['cutoff_time', 'msno']))
test = pd.get_dummies(test.drop(columns=['cutoff_time', 'msno']))

train, test = train.align(test, join='inner', axis=1)
train.shape, test.shape

### Extract the Labels

Now we can extract the labels. There are two different problems: one is a binary classification of whether or not the customer will churn during the month. The other is a regression: how many days are there until the next churn.

In [ ]:
y, test_y = np.array(train.pop('label')), np.array(test.pop('label'))

y_reg, test_y_reg = np.array(train.pop('days_to_churn')), np.array(
    test.pop('days_to_churn'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(y)
plt.title('Label Distribution')
plt.ylabel('Count')

### Fill in Missing Values

We can fill in missing values using the median of the column. As an important note, the missing test values are filled in with the median of the corresponding training feature.

In [ ]:
train = train.replace({np.inf: np.nan, -np.inf: np.nan}).\
    fillna(train.median())

In [ ]:
test = test.replace({np.inf: np.nan, -np.inf: np.nan}).\
    fillna(train.median())

In [ ]:
np.any(train.isnull()), np.any(np.isinf(train))

Now that there are no missing values and all of the values are numeric, our data is ready for machine learning. However, before we do machine learning, we need to figure out what a naive baseline would score.

# Naive Baseline

For a naive baseline, we can randomly guess that a customer has churned with the same frequence of the churns in the training data. We'll assess the predictions using a number of different metrics.

## Metrics

For an imbalanced classification problem, there are a number of metrics to consider:

* Receiver Operating Characteristic Area Under the Curve (ROC AUC): a measure between 0 and 1 comparing the performance of the classifier when predicting probabilities across a range of thresholds.
* Precision Score: number of true positives divided by the total number of positives predicted
* Recall Score: number of true positives divided by the total number of actual positives in the data
* F1 Score: Harmonic mean of precision and recall

The exact metric used and the threshold that our model needs to reach depends on the business need. We can tune the model to some extent to optimize for different metrics.

In [ ]:
np.random.seed(50)
naive_guess = np.random.binomial(1, p=np.mean(y), size=len(test_y))
naive_guess[:10], naive_guess.sum()

In [ ]:
from sklearn.metrics import (roc_auc_score, precision_score,
                             recall_score, f1_score)

print(f'Naive Baseline\n')
roc = roc_auc_score(test_y, np.repeat(np.mean(y), len(test_y)))
print(f'ROC AUC: {round(roc, 4)}')

for metric in [precision_score, recall_score, f1_score]:
    print(f'{metric.__name__}: {round(metric(test_y, naive_guess), 4)}')

We can see that these metrics are very poor! With no machine learning, we are only able to identify 2.9% of the customer churns and only 0.9% of the predicted churns were actually churns. A naive approach clearly does not provide much value.

In [ ]:
print(
    f'The percentage of churns is {100 * round(np.mean(y), 4)}% in the training data.')

# Machine Learning

Now we need to see if machine learning is up to the task of improving on these predictions. We'll start simple, using a linear model to assess if this problem is easy enough to be solved with Logistic Regression. 

(The machine learning models are implemented in [Scikit-Learn](https://sklearn.org/)). 

## Baseline Model

We can use a logistic regression in order to see baseline performance on this problem. If the logistic regression works well enough, then there is no need to move to a more complex model.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=50)

We can write a simple function to evaluate predictions. This implements the metrics used above. (A full list of metrics in Scikit-Learn can be found in [this documentation](http://scikit-learn.org/stable/modules/classes.html))

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score


def evaluate(model, train, y, test, test_y):
    """Evaluate a machine learning model on four metrics:
       ROC AUC, precision score, recall score, and f1 score.

       Returns the model and the predictions."""

    model.fit(train, y)

    # Predict probabilities and labels
    probs = model.predict_proba(test)[:, 1]
    preds = model.predict(test)

    # Calculate ROC AUC
    roc = roc_auc_score(test_y, probs)
    name = repr(model).split('(')[0]
    print(f"{name}\n")
    print(f'ROC AUC: {round(roc, 4)}')

    # Iterate through metrics
    for metric in [precision_score, recall_score, f1_score]:
        # Use .__name__ attribute to list metric
        print(f'{metric.__name__}: {round(metric(test_y, preds), 4)}')

    return model, preds


model, preds = evaluate(model, train, y, test, test_y)

The baseline model does perform slightly better than guessing in terms of ROC AUC. The precision is slightly higher than the naive baseline although the recall is much lower leading to an overall reduced f1 score (with a default threshold of 0.5 for classifying positive examples). This poor performance by the logistic regression indicates the problem of separating churn from not-churn is non-linear.

## More Complex Model

For a potentially better machine learning model, we can move to the [Random Forest Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). From the results of the logistic regression, this looks to be a non-linear problem which means we should use a model capable of learning a non-linear decision boundary.

We'll use most of the default hyperparameters but alter a few to prevent overfitting. We can also set `class_weight = 'balanced'` to try and offset the impact of such an imbalanced classification problem.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=40,
                               min_samples_leaf=50,
                               n_jobs=-1, class_weight='balanced',
                               random_state=50)

model, preds = evaluate(model, train, y, test, test_y)

The random forest performance is much better than just guessing! With no tuning, the model is able to identify 36% of the customer churns and the false positives have been reduced. This should give us confidence that we can solve this problem using machine learning.

# Model Validation

We need to inspect the model results to determine if it meets our business needs. This includes looking at the performance as well as the feature importances. We want to make sure that our model performs well, but also try and understand _why_ it performs well. 

## Precision Recall Curve

One of the best methods for tuning a model for a business need is through the precision recall curve. This shows the precision-recall tradeoff for different thresholds. Depending on the business requirement, we can change the threshold for classifying a positive example to alter the balance of true positives, false positives, false negatives, and true negatives. There will always be a tradeoff between precision and recall, but we can try to find the right balance by visually and quantitatively assessing the model.

In [ ]:
from sklearn.metrics import precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('seaborn')


def plot_precision_recall(test_y, probs, title='Precision Recall Curve', threshold_selected=None):
    """Plot a precision recall curve for predictions. 
       Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py"""

    precision, recall, threshold = precision_recall_curve(test_y, probs)
    plt.figure(figsize=(10, 8))
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall', size=18)
    plt.ylabel('Precision', size=18)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(title, size=20)
    plt.xticks(size=14)
    plt.yticks(size=14)

    if threshold_selected:
        p = precision(np.where(threshold == threshold_selected)[0])
        r = recall(np.where(threshold == threshold_selected)[0])
        plt.scatter(r, p, marker='*', size=200)
        plt.vlines(r, ymin=0, ymax=p, linestyles='--')
        plt.hlines(p, xmin=0, xmax=r, linestyles='--')

    pr = pd.DataFrame({'precision': precision[:-1], 'recall': recall[:-1],
                       'threshold': threshold})
    return pr


probs = model.predict_proba(test)[:, 1]
pr_data = plot_precision_recall(
    test_y, probs, title='Precision-Recall Curve for Random Forest')

We can query the dataframe to find the threshold required for a given precision or recall. For example, to find the threshold for a precision of 25%, we use the following code:

In [ ]:
precision_above = pr_data.loc[pr_data['precision'] >= 0.25].copy()
precision_above.sort_values('recall', ascending=False, inplace=True)
precision_above.head()

We can see that if we want a precision of 25%, then our recall will be 42.5%. This means we'll miss over 50% of the true churns in the data.

### Adjusting for the Business Requirement

Let's say we are required to have a recall of 75% in our model. This means our model finds 75% of the true churns in the data. We'll work through the rest of this notebook under this assumption. To find the threshold, we use:

In [ ]:
recall_attained = 0.75
recall_above = pr_data.loc[pr_data['recall'] >= recall_attained].copy()
recall_above.sort_values('precision', ascending=False, inplace=True)
recall_above.head()

In [ ]:
precision_attained = recall_above.iloc[0, 0]
threshold_required = recall_above.iloc[0, -1]

print(
    f'At a threshold of {round(threshold_required, 4)} the recall is {100 * recall_attained:.2f}% and the precision is {round(100 * precision_attained, 4)}%')

This means that in order to identify 75% of the actual churns, we'll have to accept that only 8.31% of the predicted positives are actually positive churns. 

In [ ]:
def plot_precision_recall(test_y, probs, title='Precision Recall Curve', threshold_selected=None):
    """Plot a precision recall curve for predictions. 
       Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py"""

    precision, recall, threshold = precision_recall_curve(test_y, probs)
    plt.figure(figsize=(10, 10))
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall', size=24)
    plt.ylabel('Precision', size=24)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(title, size=24)
    plt.xticks(size=18)
    plt.yticks(size=18)

    if threshold_selected:
        p = precision[np.where(threshold == threshold_selected)[0]]
        r = recall[np.where(threshold == threshold_selected)[0]]
        plt.scatter(r, p, marker='*', s=600, c='r')
        plt.vlines(r, ymin=0, ymax=p, linestyles='--')
        plt.hlines(p, xmin=0, xmax=r, linestyles='--')
        plt.text(r - 0.1, p + 0.15,
                 s=f'Threshold: {round(threshold_selected, 2)}', size=20, fontdict={'weight': 1000})
        plt.text(r - 0.2, p + 0.075,
                 s=f'Precision: {round(100 * p[0], 2)}% Recall: {round(100 * r[0], 2)}%', size=20,
                 fontdict={'weight': 1000})

    pr = pd.DataFrame({'precision': precision[:-1], 'recall': recall[:-1],
                       'threshold': threshold})
    return pr

In [ ]:
pr_data = plot_precision_recall(test_y, probs, title='Precision-Recall Curve for Tuned Random Forest',
                                threshold_selected=threshold_required)

## Confusion Matrix

A confusion matrix is often a useful way to visualize predictions. This shows the true values along the top row and the predicted values along the bottom row. Looking at the different cells, we can see where the model performed well and where it did not do so well.

We'll use the threshold identified above to construct the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.YlOrRd):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.style.use('bmh')
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size=22)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size=20)
    plt.yticks(tick_marks, classes, size=20)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 size=20)
    plt.grid(None)
    plt.ylabel('True label', size=22)
    plt.xlabel('Predicted label', size=22)
    plt.tight_layout()

In [ ]:
# Make predictions where probability is above threshold
preds = np.zeros(len(test_y))
preds[probs >= threshold_required] = 1

# Make and plot confusion matrix
cm = confusion_matrix(test_y, preds)
plot_confusion_matrix(cm, classes=['No Churn', 'Churn'],
                      title='Churn Confusion Matrix')

If we satisfy our business requirement, this is the best prediction of what our performance would be on new data. The model is able to identiy 75% of churned customers compared to a baseline of around 3%. The precision has increased from the baseline 1% to 8%, a relative increase of over 800%.

### Feature Importances

As one method for trying to understand how the model makes decisions, we can look at the most important features. The absolute value of the importances is not as useful as is the relative ranking of the features which is determined by how well the feature separates the classes when building the decision trees in the random forest.

In [ ]:
fi = pd.DataFrame({'importance': model.feature_importances_}, index=train.columns).\
    sort_values('importance', ascending=False)
fi.head(10)

In [ ]:
fi.iloc[:10]['importance'].plot.barh(color='r', edgecolor='k',
                                     figsize=(14, 10), linewidth=2)
ax = plt.gca()
ax.invert_yaxis()
plt.xticks(size=20)
plt.yticks(size=18)
plt.title('Most Important Features', size=28);

To see why these features are important, we can plot the distribution colored by the value of the label.
We'll look at both a kernel density estimate plot and an empirical cumulative distribution frequency diagram.

In [ ]:
# Inspiration: https://campus.datacamp.com/courses/statistical-thinking-in-python-part-1/graphical-exploratory-data-analysis?ex=12
def ecdf(x):
    n = len(x)
    x = np.sort(x)
    y = np.arange(1, n + 1) / n
    return x, y

In [ ]:
import seaborn as sns
plt.style.use('fivethirtyeight')

train_with_labels = train.copy()
train_with_labels['label'] = y
train_with_labels = train_with_labels.loc[train_with_labels[
    'TOTAL_PREVIOUS_MONTH(transactions.actual_amount_paid, transaction_date)'] < 200]

train_with_labels['TIME_SINCE_LAST(transactions.transaction_date)'] = train_with_labels[
    'TIME_SINCE_LAST(transactions.transaction_date)'] / (3600 * 24)
train_with_labels['AVG_TIME_BETWEEN(transactions.transaction_date)'] = train_with_labels[
    'AVG_TIME_BETWEEN(transactions.transaction_date)'] / (3600 * 24)

# Iterate through features
for feature in ['TOTAL_PREVIOUS_MONTH(transactions.actual_amount_paid, transaction_date)',
                'TIME_SINCE_LAST(transactions.transaction_date)',
                'AVG_TIME_BETWEEN(transactions.transaction_date)',
                'SUM(transactions.payment_plan_days WHERE is_auto_renew = 1)',
                'LAST(transactions.payment_method_id)']:
    plt.figure(figsize=(20, 8))

    # Iterate through values of the label
    for label, grouped in train_with_labels.groupby('label'):
        # Plot the distribution of the feature
        sns.kdeplot(grouped[feature].dropna(),
                    label='Churned' if label == 1 else 'No Churn')
    # Plot labeling
    plt.ylabel('Density')
    plt.xlabel(f'{feature.capitalize()}')
    plt.title(f'Distribution of {feature.capitalize()}', size=28)
    plt.legend(prop={'size': 20})

In [ ]:
# Iterate through features
for feature, name in zip(['TOTAL_PREVIOUS_MONTH(transactions.actual_amount_paid, transaction_date)',
                'TIME_SINCE_LAST(transactions.transaction_date)'], 
                         ['Total Spending in Month before Cutoff Time', 
                           'Time Since Last Transaction']):
    plt.figure(figsize=(20, 8))

    # Iterate through values of the label
    for label, grouped in train_with_labels.groupby('label'):
        # Plot the distribution of the feature
        sns.kdeplot(grouped[feature].dropna(),
                    label='Churned' if label == 1 else 'No Churn')
    # Plot labeling
    plt.ylabel('Density')
    plt.xlabel(f'{name}')
    plt.title(f'Distribution of {name}', size=28)
    plt.legend(prop={'size': 20})

There are significant differences between those customers who churned and those who did not. Most noticeably, the customers who churned had less activity in the previous month.

In [ ]:
# Iterate through features
for feature, name in zip(['TOTAL_PREVIOUS_MONTH(transactions.actual_amount_paid, transaction_date)',
                          'TIME_SINCE_LAST(transactions.transaction_date)',
                          'AVG_TIME_BETWEEN(transactions.transaction_date)'],
                         ['Total Spent in Previous Month',
                          'Time Since Last Transaction',
                          'Average Time Between Transactions']):
    plt.figure(figsize=(10, 6))

    # Iterate through values of the label
    for label, grouped in train_with_labels.groupby('label'):
        xs, ys = ecdf(grouped[feature])

        # Plot the distribution of the feature
        plt.plot(xs, ys, marker='.', ms=10,
                 label='Churned' if label == 1 else 'No Churn')

    # Plot labeling
    plt.ylabel('Percentile')
    plt.xlabel(f'{name.capitalize()}')
    plt.title(f'ECDF of {name.capitalize()}', size=28)
    plt.legend(prop={'size': 20})

# Make Predictions

Now we will use the trained model to make predictions on the hold-out testing set. We'll can make predictions in terms of probabilities and then threshold them to labels using our selected threshold.

In [ ]:
new_probs = model.predict_proba(test)[:, 1]


oos_ids = list(feature_matrix.loc[feature_matrix['cutoff_time'] >= split_date, 'msno'])
oos_cutoff_time = list(feature_matrix.loc[feature_matrix['cutoff_time'] >= split_date, 'cutoff_time'])

prediction_df = pd.DataFrame({'msno': oos_ids, 'cutoff_time': oos_cutoff_time,
                              'probability': new_probs})
prediction_df['prediction'] = prediction_df['probability'] > threshold_required
prediction_df.sample(10)

In [ ]:
prediction_df['probability'].plot.hist(bins=20, edgecolor='k')
plt.axvline(x=threshold_required, color='r', linewidth=2)
plt.title('Distribution of Predicted Probabilities')

The model is fairly confident that most of the predictions are not churns. Moreover, there are a number of probabilities that are exactly the same as can be seen in the [empirical cumulative density function](https://en.wikipedia.org/wiki/Empirical_distribution_function). 

In [ ]:
xs, ys = ecdf(prediction_df['probability'])
plt.plot(xs, ys, marker='.')
plt.title('ECDF of Predicted Probobabilities')
plt.axvline(x=threshold_required, color='r', linewidth=2)
plt.xlabel('Probability')
plt.ylabel('Percentile')

# Business Value Analysis

Using the metrics from the model, we can conduct an analysis of the business value of our solution. We'll use the precision and recall along with a few assumptions.

* Typical plan price = 150 (NTD)
* Reduced plan price = 130 (NTD)
* Recall = 75%
* Precision = 8.31%
* Conversion Rate = 75%

$$\text{Recall} = \frac{TP}{TP + FN}$$

$$\text{Precision} = \frac{TP}{TP + FP}$$

We can scale our analysis to the entire dataset since we only used a subset of it for modeling. We'll assume we are able to achieve the same performance on the entire dataset, which is reasonable because the accuracy of a model generally improves as the amount of data used increases.

In [ ]:
plan_price = 150
r_plan_price = 130
recall = recall_attained
precision = precision_attained
conversion_rate = 0.75

# Find total number of members
n_members = pd.read_csv('../data/members_v3.csv', usecols = ['msno']).shape[0]
monthly_revenue = n_members * plan_price
churn_rate = np.mean(feature_matrix['label'])

churns = int(churn_rate * n_members)
# Find the typical monthly revenue lost to churned customers
revenue_lost_churns = n_members * churn_rate * plan_price
print(f'Typical monthly revenue lost to {churns} churned customers = ${revenue_lost_churns:,.2f} (NTD).')
print(f'Typical total monthly revenue = ${monthly_revenue:,.2f}.')
print(f'Churns losses represent {100 * (revenue_lost_churns / monthly_revenue):.2f}% of monthly revenue.')

In [ ]:
# Find stats for entire dataset
true_positives = int(churns * recall)
false_negatives = int(churns - true_positives)
false_positives = int((true_positives * (1 - precision)) / precision)

print(f'True positives: {true_positives}; False negatives: {false_negatives}; False positives: {false_positives}.')

As a sanity check, we'll make sure the precision and recall metrics come out right.

In [ ]:
(true_positives) / (true_positives + false_positives)

In [ ]:
(true_positives) / (true_positives + false_negatives)

Things look right here.

In [ ]:
revenue_lost_false_positives = false_positives * (plan_price - r_plan_price)
revenue_lost_false_negatives = false_negatives * plan_price
revenue_recouped_true_positives = conversion_rate * (true_positives * r_plan_price)

print(f'Cost from false positives = ${revenue_lost_false_positives:,.2f} (NTD); Cost from false negatives = ${revenue_lost_false_negatives:,.2f} (NTD)')
print(f'Revenue recouped from true positives = ${revenue_recouped_true_positives:,.2f} (NTD).')

In [ ]:
total_effect = revenue_lost_churns - (-revenue_recouped_true_positives + revenue_lost_false_positives + revenue_lost_false_negatives)
us_dollars = total_effect * 0.033
print(f'Total Effect of identifying churns ${total_effect:,.2f} (NTD) = ${us_dollars:,.2f} (USD).')
print(f'This represents {100 * (total_effect / revenue_lost_churns):.2f}% of the losses due to churns.')

Our model has successfully solved the business problem, recouping over 10% of the losses due to churns. This does require a number of assumptions, but with additional refinement, our model could probably deliver even greater value.

# Automated Model Optimization Using TPOT

If we are not pleased with the model results using an off-the-shelf algorithm from Scikit-Learn, there are a number of libraries for searching for the best model without any manual intervention. One of the easiest-to-se libraries is known as TPOT. This will search through hundreds of machine learning models, using evolutionary algorithms to guide the discovery process.

To evaluate the models, we'll use a TimeSeriesSplit. This makes three splits of the data based on the indexes so we need to ensure that our data is sorted by time (already done). 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=3)

Now we make a `TPOTClassifier` object and pass in a few parameters. This particular use case will search 100 models, using `f1` scoring, the TimeSeriesSplit for cross validation, and taking advantage of all the cores on our machine.

In [ ]:
from tpot import TPOTClassifier

# Make tpot pipeline
tpot_pipeline = TPOTClassifier(generations=10, population_size=10,
                               cv=tss, scoring='f1',
                               n_jobs=-1, verbosity=2,
                               random_state=50)

In [ ]:
tpot_pipeline.fit(train, y)

In [ ]:
tpot_pipeline.fitted_pipeline_

In [ ]:
tpot_pipeline.export('best_pipeline_new.py')

## Best Pipeline

The following code shows the best pipeline exported by TPOT. We can just copy the code, train a model, and make predictions. 

In [ ]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
import IPython

with open('best_pipeline.py') as f:
    code = f.read()

formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
    formatter.get_style_defs('.highlight'),
    highlight(code, PythonLexer(), formatter)))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

exported_pipeline = ExtraTreesClassifier(bootstrap=False,
                                         criterion="entropy",
                                         max_features=0.1,
                                         min_samples_leaf=5,
                                         min_samples_split=8,
                                         n_estimators=100,
                                         random_state=50,
                                         n_jobs=-1)

In [ ]:
model, preds = evaluate(exported_pipeline, train, y, test, test_y)

Now we can do the same procedure as before, looking at the precision recall curve and finding the threshold needed for a recall of 75%.

In [ ]:
probs = model.predict_proba(test)[:, 1]
pr_data = plot_precision_recall(
    test_y, probs, title='Precision-Recall Curve for Optimized Pipeline')

In [ ]:
recall_above = pr_data.loc[pr_data['recall'] >= recall_attained].copy()
recall_above.sort_values('precision', ascending=False, inplace=True)
recall_above.head()

In [ ]:
precision_attained = recall_above.iloc[0, 0]
threshold_required = recall_above.iloc[0, -1]

print(
    f'At a threshold of {round(threshold_required, 4)} the recall is {100 * recall_attained:.2f}% and the precision is {round(100 * precision_attained, 4)}%')

The Random Forest Classifier built by hand does basically the sameas the TPOT optimized model. Running the optimization for longer may result in a better model. However, in terms of time investment, a better option is probably to spend more time on feature engineering. 

In [ ]:
preds = np.zeros(len(test_y))
preds[probs >= threshold_required] = 1

cm = confusion_matrix(test_y, preds)
plot_confusion_matrix(cm, classes=['No Churn', 'Churn'],
                      title='Optimized Model Confusion Matrix')

# Conclusions

Our final tuned model was able to achieve a recall of 75% and a precision of 8.31%, a significant increase over the naive baseline of 3.5% recall and 1.0% precision. 

Overall, the final metrics from modeling are:

| Model                                     | ROC AUC | Recall | Precision | F1 Score |
|-------------------------------------------|---------|--------|-----------|----------|
| Naive Baseline (no ml)                    | 0.5     | 3.47%  | 1.04%     | 0.016   |
| Logistic Regression                       | 0.577   | 0.51%  | 2.91%     | 0.009   |
| Random Forest Default                     | 0.929   | 65.2%  | 14.7%     | 0.240   |
| Random Forest Tuned for 75% Recall        | 0.929   | 75%    | 8.31%     | 0.150    |
| Auto-optimized Model                      | 0.927   | 2.88%  | 64.4%     | 0.055   |
| Auto-optimized Model Tuned for 75% Recall | 0.927   | 75%    | 9.58%     | 0.170    |

With these metrics, we can conclude that machine learning has solved the business problem of identifying customers that will churn in the next month. We can now use this model to predict customers at risk for churning in the future. An analysis of the business payback also showed that the final model delivered significant value. The predictions would then be handed off to the customer engagement team to hopefully reduce the number of churns.

The framework for solving this problem - and for solving any problem with machine learning - was:

1. Prediction Engineering: define the business objective, translate into a machine learning task, and create a set of labeled historical examples with cutoff time from the data.
2. Feature Engineering: use the label times to automatically build hundreds of relevant and valid features for each label.
3. Modeling: use a machine learning algorithm implemented in common Python libraries to train a model to predit the labels from the features. Validate and tune model for the business need and then make predictions on new data.

![](../images/Framework.png)

This approach solves a number of problems: it standardizes the traditionally ad-hoc process of solving a problem with machine learning and it is general so that nearly the same code can be used for multiple prediction problems on the same dataset. The benefits of machine learning have been limited to a few companies both because of the lack of a shared language for expressing and solving problems, and because each solution requires custom code and must be completely rebuilt for a different prediction problem. By codifying the processes needed to solve problems with machine learning, we aim to make it easier for companies to use this transformational technology. 

Furthermore, the framework lets data scientists fill in the details with existing tools allowing for rapid development and deployment of model pipelines. The achievements of machine learning are already impressive, but with a scaffolding in place the benefits can be extended to a broader population.
